In [ ]:
# @title **Welcome**
import ipywidgets as widgets

# Creating the heading with Andika font and larger size
heading = widgets.HTML("<h1 style='font-family: Andika, sans-serif; font-size: 36px;'><span style='color: #ff0066;'>U</span><span style='color: #ff6f00;'>X</span><span style='color: #ffcd00;'>E</span><span style='color: #4caf50;'>L</span> <span style='color: #2196f3;'>V</span><span style='color: #9c27b0;'>i</span><span style='color: #ff5722;'>s</span><span style='color: #e91e63;'>i</span><span style='color: #009688;'>o</span><span style='color: #ffeb3b;'>n</span> <span style='color: #3f51b5;'>2</span><span style='color: #ff9800;'>0</span><span style='color: #795548;'>2</span><span style='color: #4caf50;'>4</span><br><span style='font-size: 17px;'>by Vishal Sharma</span><br><span style='font-size: 18px;'>Enhance Your Videos with ESRGAN</span></h1>")

# Displaying the heading
display(heading)

HTML(value="<h1 style='font-family: Andika, sans-serif; font-size: 36px;'><span style='color: #ff0066;'>U</spa…

# **Start**

In [ ]:
import torch
from tqdm import tqdm
import cv2
import os
import subprocess

assert torch.cuda.is_available(), "GPU not detected. Please ensure CUDA is installed."

# Function to display progress bar
def progress_bar(iterable, desc=None):
    if hasattr(tqdm, '_instances'):
        tqdm._instances.clear()  # clear if it exists
    return tqdm(iterable, desc=desc, ncols=100)

# Hiding execution attributes
#@title Installation Process { display-mode: "form" }

!git clone https://github.com/xinntao/Real-ESRGAN.git > /dev/null 2>&1
%cd Real-ESRGAN

# Install required packages
!pip install -q torch==2.0.1 torchvision==0.15.2
!pip install -q basicsr facexlib gfpgan ffmpeg ffmpeg-python

# Install other requirements
!pip install -q -r requirements.txt
!python setup.py develop

# Installation progress bar
with progress_bar(range(100), desc="Installation") as pbar:
    for _ in pbar:
        # Simulate installation process
        pass

# Installation complete message
print("Installation complete.")

/content/Real-ESRGAN/Real-ESRGAN/Real-ESRGAN/Real-ESRGAN
/usr/local/lib/python3.10/dist-packages/setuptools/__init__.py:84: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements should be satisfied by a PEP 517 installer.
        If you are using pip, you can try `pip install --use-pep517`.
        ********************************************************************************

!!
  dist.fetch_build_eggs(dist.setup_requires)
running develop
/usr/local/lib/python3.10/dist-packages/setuptools/command/develop.py:40: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See htt

Installation: 100%|███████████████████████████████████████████| 100/100 [00:00<00:00, 537731.28it/s]

Installation complete.


#**After Completing Installation**

In [ ]:
# @title **Upload Video Files**
# Import necessary libraries
import os
from google.colab import drive
import ipywidgets as widgets
from IPython.display import display, HTML

# Create input directory if it doesn't exist
if not os.path.exists('/content/'):
    os.makedirs('/content/')

# Function to check if Google Drive is mounted
def is_drive_mounted():
    return os.path.exists("/content/gdrive/")

# Function to display mount status indicator with color
def display_mount_status():
    if is_drive_mounted():
        status_indicator = '<span style="color:green">&#x25CF;</span>'
    else:
        status_indicator = '<span style="color:red">&#x25CF;</span>'
    display(HTML(f"<b>Google Drive Mount Status:</b> {status_indicator}"))

# Function to mount Google Drive
def mount_drive(button):
    drive.mount('/content/gdrive')
    display_mount_status()

# Function to handle file upload
def upload_files(button):
    total_files = len(file_drop_box.value)
    for i, (file_name, file_info) in enumerate(file_drop_box.value.items(), 1):
        if file_name.endswith('.mp4'):
            content = file_info['content']
            with open(os.path.join('/content/', file_name), 'wb') as f:
                f.write(content)
        progress_bar.value = i / total_files
        progress_bar.description = f"Progress: {int(progress_bar.value * 100)}%"
    display_popup_message("Upload Successful!")

# Function to display popup message
def display_popup_message(message):
    popup_message.value = message

# Function to refresh the page
def refresh_page(button):
    display.clear_output(wait=True)
    main()

# Create UI elements
mount_button = widgets.Button(description="Mount Google Drive")
upload_button = widgets.Button(description="Upload", button_style="success")
progress_bar = widgets.FloatProgress(value=0.0, min=0.0, max=1.0, description="Progress:")
popup_message = widgets.Text(value="", disabled=True)
file_drop_box = widgets.FileUpload(accept='.mp4', multiple=True)

# Display mount status indicator
display_mount_status()

# Display UI elements
display(file_drop_box)
display(mount_button)
display(upload_button)
display(progress_bar)
display(popup_message)

# Assign event handlers
mount_button.on_click(mount_drive)
upload_button.on_click(upload_files)
import ipywidgets as widgets

# Create a divider widget
divider = widgets.HTML("<hr/>")

# Display the divider
display(divider)

FileUpload(value={}, accept='.mp4', description='Upload', multiple=True)

Button(description='Mount Google Drive', style=ButtonStyle())

Button(button_style='success', description='Upload', style=ButtonStyle())

FloatProgress(value=0.0, description='Progress:', max=1.0)

Text(value='', disabled=True)

HTML(value='<hr/>')

In [ ]:
# @title **Run Upscaled**
import os
import cv2
import subprocess
import ipywidgets as widgets
from IPython.display import display
import re

# Function to fetch files from content directory with jpg or mp4 format
def fetch_files(folder_path):
    files = []
    for file in os.listdir(folder_path):
        if file.endswith(".jpg") or file.endswith(".mp4"):
            files.append(file)
    return files

# Dropdown menu for selecting the video path
video_path_dropdown = widgets.Dropdown(
    options=fetch_files("/content/"),
    description='video path:',
    disabled=False,
)

# Radio buttons for selecting the resolution
resolution_radio = widgets.RadioButtons(
    options=["FHD (1920 x 1080)", "2k (2560 x 1440)", "4k (3840 x 2160)", "2 x original", "3 x original", "4 x original"],
    description='Resolution:',
    disabled=False
)

# Dropdown menu for selecting the model
model_dropdown = widgets.Dropdown(
    options=["RealESRGAN_x4plus", "RealESRGAN_x4plus_anime_6B", "realesr-animevideov3"],
    description='Model:',
    disabled=False,
)

# Button to start the ESRGAN process
start_button = widgets.Button(description="Start")

# Output widget to display success messages
output = widgets.Output()

# Progress bar widget
progress_bar = widgets.FloatProgress(
    min=0,
    max=1.0,
    description='Progress:',
    bar_style='info', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal'
)

# Container to display progress bar
progress_box = widgets.HBox([progress_bar])

# Function to execute ESRGAN process
def start_esrgan_process(_):
    with output:
        output.clear_output()
        video_path = f"/content/{video_path_dropdown.value}"
        output_dir = "/content/"
        resolution = resolution_radio.value
        model = model_dropdown.value

        assert os.path.exists(video_path), "Video file does not exist"

        video_capture = cv2.VideoCapture(video_path)
        video_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
        video_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

        final_width = None
        final_height = None
        aspect_ratio = float(video_width / video_height)

        # Get output resolutions
        match resolution:
            case "FHD (1920 x 1080)":
                final_width = 1920
                final_height = 1080
            case "2k (2560 x 1440)":
                final_width = 2560
                final_height = 1440
            case "4k (3840 x 2160)":
                final_width = 3840
                final_height = 2160
            case "2 x original":
                final_width = 2 * video_width
                final_height = 2 * video_height
            case "3 x original":
                final_width = 3 * video_width
                final_height = 3 * video_height
            case "4 x original":
                final_width = 4 * video_width
                final_height = 4 * video_height

        if aspect_ratio == 1.0 and "original" not in resolution:
            final_height = final_width

        if aspect_ratio < 1.0 and "original" not in resolution:
            temp = final_width
            final_width = final_height
            final_height = temp

        scale_factor = max(final_width / video_width, final_height / video_height)
        isEven = int(video_width * scale_factor) % 2 == 0 and int(video_height * scale_factor) % 2 == 0

        # scale_factor needs to be even
        while isEven == False:
            scale_factor += 0.01
            isEven = int(video_width * scale_factor) % 2 == 0 and int(video_height * scale_factor) % 2 == 0

        print(f"Upscaling from {video_width}x{video_height} to {final_width}x{final_height}, scale_factor={scale_factor}")

        progress_bar.value = 0  # Reset progress bar

        # Start ESRGAN process
        process = subprocess.Popen(['python', 'inference_realesrgan_video.py', '-n', model, '-i', video_path, '-o', output_dir, '--outscale', str(scale_factor)], stderr=subprocess.PIPE, universal_newlines=True)

        for line in process.stderr:
            if 'inference' in line:
                progress_percentage = re.search(r'(\d+)%', line).group(1)
                progress_bar.value = int(progress_percentage) / 100

        # Wait for the process to finish
        process.wait()

        video_name_with_ext = os.path.basename(video_path)
        video_name = video_name_with_ext.replace(".mp4", "")
        upscaled_video_path = f"{output_dir}{video_name}_out.mp4"
        final_video_name = f"{video_name}_upscaled_{final_width}_{final_height}.mp4"
        final_video_path = os.path.join(output_dir, final_video_name)

        # crop to fit
        if "original" not in resolution:
            print("Cropping to fit...")
            command = f"ffmpeg -loglevel error -hwaccel cuda -y -i '{upscaled_video_path}' -c:v h264_nvenc -filter:v  'crop={final_width}:{final_height}:(in_w-{final_width})/2:(in_h-{final_height})/2' -c:v libx264 -pix_fmt yuv420p '{final_video_path}'"
            subprocess.run(command, shell=True)

        print(f"Upscaled video saved to: {final_video_path}")

        # Display success message
        print("ESRGAN process completed successfully!")

        # save to drive
        if mount_drive:
            drive_folder = "MyDrive/Upscaled Videos (REAL-ESRGAN)"
            save_directory_drive = f"/content/gdrive/{drive_folder}"
            os.makedirs(save_directory_drive, exist_ok=True)

            command = f"'{final_video_path}' '{save_directory_drive}'/'{final_video_name}'"
            subprocess.run(command, shell=True)
            print(f"Saved to drive: /{drive_folder}/{final_video_name}")

        !rm "{upscaled_video_path}"

# Event handler for start button click
start_button.on_click(start_esrgan_process)

# Display interface
display(video_path_dropdown)
display(resolution_radio)
display(model_dropdown)
display(start_button)
display(output)
display(progress_box)

import ipywidgets as widgets

# Define the content for the popup note
note_content = """
This process may take some time depending on the size and resolution of your video.
The progress bar will show the progress of the upscaling process.
Once completed, the upscaled video will be saved in the same directory as the original video with '_upscaled' appended to its name.
"""

# Create a ToggleButton to show/hide the popup note
info_button = widgets.ToggleButton(value=False, description="ℹ️ Info", button_style='')

# Create an Output widget to display the note content
note_output = widgets.Output(layout={'border': 'none', 'padding': '10px'})

# Function to toggle the visibility of the note content
def toggle_note_visibility(change):
    with note_output:
        note_output.clear_output()
        if change.new:
            print(note_content)

# Assign the toggle_note_visibility function to the observe event of the ToggleButton
info_button.observe(toggle_note_visibility, 'value')

# Display the ToggleButton and the Output widget inside a VBox
display(widgets.VBox([info_button, note_output]))

# Style the button to be black by default and green when pressed
info_button.add_class('toggle_button_black')
info_button.style.button_color = 'black'
info_button.style.description_width = 'initial'

# Add custom CSS to change button color when pressed
css = """
.toggle_button_black > .p-Widget > .p-Button > .p-Button-description:active {
    color: white !important;
    background-color: limegreen !important;
}
"""
widgets.HTML(value=f"<style>{css}</style>")

import ipywidgets as widgets
from IPython.display import display, HTML

import ipywidgets as widgets

# Create a divider widget
divider = widgets.HTML("<hr/>")

# Display the divider
display(divider)

Dropdown(description='video path:', options=('1000004550.mp4',), value='1000004550.mp4')

RadioButtons(description='Resolution:', options=('FHD (1920 x 1080)', '2k (2560 x 1440)', '4k (3840 x 2160)', …

Dropdown(description='Model:', options=('RealESRGAN_x4plus', 'RealESRGAN_x4plus_anime_6B', 'realesr-animevideo…

Button(description='Start', style=ButtonStyle())

Output()

HTML(value='<hr/>')

In [ ]:
# @title **File Manager**
import os
import shutil
import ipywidgets as widgets
from IPython.display import display, clear_output

# Function to fetch files from the content directory
def fetch_files(folder_path):
    files = [file for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]
    return files

# Function to delete selected files
def delete_selected_files(_):
    with output:
        output.clear_output()
        if toggle_radio.value == "Transfer to GDrive":
            for file in file_dropdown.value:
                file_path = os.path.join("/content", file)
                try:
                    os.remove(file_path)
                    print(f"Successfully deleted '{file}' from Content Directory!")
                except Exception as e:
                    print(f"Error deleting '{file}' from Content Directory: {e}")
            refresh_file_list()
        elif toggle_radio.value == "Transfer to Local":
            for file in gdrive_file_dropdown.value:
                file_path = os.path.join("/content/gdrive/MyDrive", file)
                try:
                    os.remove(file_path)
                    print(f"Successfully deleted '{file}' from Google Drive!")
                except Exception as e:
                    print(f"Error deleting '{file}' from Google Drive: {e}")
            refresh_gdrive_file_list()

# Function to transfer selected files
def transfer_selected_files(_):
    with output:
        output.clear_output()
        progress_bar.value = 0
        if toggle_radio.value == "Transfer to GDrive":
            selected_files = file_dropdown.value
            total_files = len(selected_files)
            for file in selected_files:
                source_path = os.path.join("/content", file)
                destination_path = "/content/gdrive/MyDrive/"
                try:
                    shutil.move(source_path, destination_path)
                    progress_bar.value += 100 / total_files
                    print(f"Successfully transferred '{file}' to Google Drive!")
                except Exception as e:
                    print(f"Error transferring '{file}' to Google Drive: {e}")
            refresh_file_list()
            success_label.value = "Transfer to Google Drive complete!"
        elif toggle_radio.value == "Transfer to Local":
            selected_files = gdrive_file_dropdown.value
            total_files = len(selected_files)
            for file in selected_files:
                source_path = os.path.join("/content/gdrive/MyDrive", file)
                destination_path = "/content/"
                try:
                    shutil.move(source_path, destination_path)
                    progress_bar.value += 100 / total_files
                    print(f"Successfully transferred '{file}' to Content Directory!")
                except Exception as e:
                    print(f"Error transferring '{file}' to Content Directory: {e}")
            refresh_gdrive_file_list()
            success_label.value = "Transfer to Content Directory complete!"

# Function to refresh file list from content directory
def refresh_file_list(_=None):
    file_dropdown.options = fetch_files("/content/")

# Function to refresh file list from Google Drive
def refresh_gdrive_file_list(_=None):
    gdrive_file_dropdown.options = fetch_files("/content/gdrive/MyDrive")

# Dropdown menu for content directory file selection
file_dropdown = widgets.SelectMultiple(
    options=fetch_files("/content/"),
    description='Content Files:',
    disabled=False
)

# Dropdown menu for Google Drive file selection
gdrive_file_dropdown = widgets.SelectMultiple(
    options=fetch_files("/content/gdrive/MyDrive"),
    description='GDrive Files:',
    disabled=False
)

# Button to delete selected files
delete_button = widgets.Button(description="Delete Selected", button_style='danger')

# Button to refresh file list
refresh_button = widgets.Button(description="Refresh Files")

# Radio button for transfer direction
toggle_radio = widgets.RadioButtons(
    options=['Transfer to GDrive', 'Transfer to Local'],
    description='Transfer Direction:',
    disabled=False,
    style={'description_width': 'initial'}
)

# Button to transfer selected files
transfer_button = widgets.Button(description="Transfer", button_style='success')

# Progress bar
progress_bar = widgets.FloatProgress(
    value=0,
    min=0,
    max=100,
    description='Progress:',
    bar_style='info',
    orientation='horizontal'
)

# Success message
success_label = widgets.Label(value="")

# Output widget to display success or error messages
output = widgets.Output()

# Event handlers
delete_button.on_click(delete_selected_files)
refresh_button.on_click(refresh_file_list)
transfer_button.on_click(transfer_selected_files)

# Display interface
toggle_radio.style.button_color = 'limegreen'
display(widgets.VBox([file_dropdown, gdrive_file_dropdown, refresh_button, delete_button, toggle_radio, transfer_button, progress_bar, success_label, output]))
import ipywidgets as widgets

# Create a divider widget
divider = widgets.HTML("<hr/>")

# Display the divider
display(divider)

HTML(value='<hr/>')

In [ ]:
# @title **Download Processed Files**
import os
import ipywidgets as widgets
from IPython.display import display, HTML

# Mount Google Drive
from google.colab import drive

# Function to check if Google Drive is mounted
def is_drive_mounted():
    return os.path.exists("/content/gdrive/")

# Function to display mount status indicator
def display_mount_status():
    if is_drive_mounted():
        status_indicator = '<span style="color:green">&#x25CF;</span>'
    else:
        status_indicator = '<span style="color:red">&#x25CF;</span>'
    display(HTML(f"Google Drive Mount Status: {status_indicator}"))

# Check and display mount status
display_mount_status()

# Define the directory to fetch files from
fetch_files_dir = "/content/gdrive/MyDrive/"

# Function to fetch files from the specified directory
def fetch_files():
    if not os.path.exists(fetch_files_dir):
        os.makedirs(fetch_files_dir, exist_ok=True)
    file_options = [f for f in os.listdir(fetch_files_dir) if os.path.isfile(os.path.join(fetch_files_dir, f))]
    return file_options

# Dropdown menu for selecting files
file_dropdown = widgets.Dropdown(description="Select file:")
file_dropdown.options = fetch_files()

# Refresh button
refresh_button = widgets.Button(description="Refresh", button_style="info")

# Function to handle refresh button click
def on_refresh_button_click(b):
    file_dropdown.options = fetch_files()

# Assigning click event to the refresh button
refresh_button.on_click(on_refresh_button_click)

# Green button to get Gdrive link
get_gdrive_link_button = widgets.Button(description="Get Gdrive Link", button_style="success")

# Output widget for displaying link
output_link = widgets.Output()

# Function to handle get Gdrive link button click
def on_get_gdrive_link_button_click(b):
    selected_file = file_dropdown.value
    file_path = os.path.join(fetch_files_dir, selected_file)
    file_id = get_file_id(file_path)
    link = f"https://drive.google.com/file/d/{file_id}/view?usp=drivesdk"
    with output_link:
        output_link.clear_output()
        display(HTML(f'<div style="margin-top: 10px;"><a href="{link}" target="_blank"><button style="background-color: #4CAF50; color: white; padding: 10px 24px; border: none; cursor: pointer; border-radius: 8px;">Gdrive Link</button></a></div>'))

# Assigning click event to the get Gdrive link button
get_gdrive_link_button.on_click(on_get_gdrive_link_button_click)

# Display widgets
display(HTML("<h3 style='margin-bottom: 10px;'>Download Files</h3>"))
display(file_dropdown)
display(widgets.HBox([refresh_button, get_gdrive_link_button]))
display(HTML("<hr style='margin-top: 20px;'>"))
display(output_link)

# Function to get file ID from file path
def get_file_id(file_path):
    file_name = os.path.basename(file_path)
    file_id = file_name.split('.')[0]
    return file_id

Dropdown(description='Select file:', options=(), value=None)

Output()

In [ ]:

# @title **About**
from IPython.display import HTML, display

def display_about_info(info_text):
    html_code = f"""
    <div style="color: white; text-align: center; padding: 10px; font-size: 16px; font-family: Andika, sans-serif;">
        {info_text}
    </div>
    """
    display(HTML(html_code))

# About information text
about_info_text = """
<div style="color: white; text-align: center; padding: 10px; font-size: 16px; font-family: Andika, sans-serif;">
    <h2 style="font-weight: bold;">🚩 जय श्री राम 🚩</h2>
    <h1 style="font-size: 36px;">U<span style="color: #ff0066;">X</span><span style="color: #ff6f00;">E</span><span style="color: #ffcd00;">L</span> V<span style="color: #4caf50;">i</span><span style="color: #2196f3;">s</span><span style="color: #9c27b0;">i</span><span style="color: #ff5722;">o</span><span style="color: #e91e63;">n</span> 2024</h1>
    <h4>Thanks to ESRGAN</h4>
    <br>
    <a href="https://colab.research.google.com/drive/1ch7vpHFFdkc4W48xDO3bgLMzhuGMgd8-?usp=sharing" target="_blank">
        <button style="background-color: green; color: white; padding: 20px 40px; font-size: 24px; border: none; border-radius: 10px;">Check This Unlimited AI High Quality Image Generator</button>
    </a>
    <br><br>
    <h1 style="font-size: 36px;">U<span style="color: #ff0066;">X</span><span style="color: #ff6f00;">E</span><span style="color: #ffcd00;">L</span> F<span style="color: #4caf50;">o</span><span style="color: #2196f3;">o</span><span style="color: #9c27b0;">o</span><span style="color: #ff5722;">o</span><span style="color: #e91e63;">c</span><span style="color: #009688;">u</span><span style="color: #ffeb3b;">s</span> 2024</h1>
    <br>
    <p style="color: white; font-size: 20px;">Introducing UXEL Fooocus 2024, the ultimate AI image generation tool now available via Gradio.live. With pre-added single download link models including 14 Base Models, 4 Anime Models, and 12 NSFW 18+ Uncensored Models, the possibilities are limitless. Experience the power of stable diffusion 1.0 as you effortlessly create stunning AI-generated images tailored to your needs. Explore additional models and enhancers with a simple click, all within the intuitive Gradio.live interface. Elevate your creativity and explore new horizons with UXEL Fooocus 2024. Try it now and unlock a world of imagination!</p>
</div>
"""

# Display the "About" information
display_about_info(about_info_text)